In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import plotly.express as px
import statsmodels.api as sm

In [117]:
mri = pd.read_csv("Imp_Files/UCSFFSX_mri_all.csv")

In [118]:
regions_dict = pd.read_csv("Files/mri_regional_dict_proper_all.csv")
regions_dict

,ADNI Code,Description,Region
0,ST1SV,N mm3 Volume (aseg.stat) of,Brainstem
1,ST2SV,N mm3 Volume (aseg.stat) of,CorpusCallosumAnterior
2,ST3SV,N mm3 Volume (aseg.stat) of,CorpusCallosumCentral
3,ST4SV,N mm3 Volume (aseg.stat) of,CorpusCallosumMidAnterior
4,ST5SV,N mm3 Volume (aseg.stat) of,CorpusCallosumMidPosterior
...,...,...,...
368,ST153SV,N mm3 Volume (aseg.stat) of,SubcorticalGM
369,ST155SV,N mm3 Volume (aseg.stat) of,SupraTentorial
370,ST127SV,N mm3 Volume (aseg.stat) of,ThirdVentricle
371,ST154SV,N mm3 Volume (aseg.stat) of,TotalGM


In [119]:
mapping_dict = dict(zip(regions_dict['ADNI Code'], regions_dict['Region']))

mri.rename(columns=mapping_dict, inplace=True)

mri.head()

,RID,VISCODE,EXAMDATE,VERSION,LONISID,FLDSTRENG,LONIUID,IMAGEUID,RUNDATE,STATUS,...,CorticalGM,LeftCorticalWM,RightCorticalWM,CorticalWM,SubcorticalGM,TotalGM,SupraTentorial,COLPROT,IMAGETYPE,HIPPOQC
0,2,sc,2005-08-26,2009-05-14,1945.0,1.5,9107.0,35475,2009-05-14,complete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,sc,2005-09-01,2009-05-11,1948.0,1.5,9127.0,32237,2009-05-11,complete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,m06,2006-03-13,2009-06-26,4081.0,1.5,15240.0,31863,2009-06-26,complete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,m12,2006-09-12,2009-05-27,5186.0,1.5,19096.0,35576,2009-05-27,complete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,m24,2007-09-12,2009-09-28,11918.0,1.5,39296.0,88252,2009-09-28,complete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
mri = mri[mri['STATUS']=='complete']
columns_to_keep = ['VISCODE2', 'RID'] + list(mapping_dict.values())
mri = mri[columns_to_keep]

In [121]:
mri.sort_values(by=['RID', 'VISCODE2'])

,VISCODE2,RID,Brainstem,CorpusCallosumAnterior,CorpusCallosumCentral,CorpusCallosumMidAnterior,CorpusCallosumMidPosterior,CorpusCallosumPosterior,CorticalGM,CorticalWM,...,RightUnknown,RightUnknown,RightVentralDC,RightVessel,RightWMHypoIntensities,SubcorticalGM,SupraTentorial,ThirdVentricle,TotalGM,WMHypoIntensities
0,sc,2,23003.0,810.0,362.0,432.0,404.0,1125.0,NaN,NaN,...,1.164,1.559,4135.0,77.0,NaN,NaN,NaN,3644.0,NaN,7426.0
2,m06,3,23784.0,774.0,378.0,358.0,314.0,701.0,NaN,NaN,...,0.847,1.169,3951.0,220.0,NaN,NaN,NaN,3036.0,NaN,44708.0
3,m12,3,23759.0,763.0,369.0,323.0,307.0,721.0,NaN,NaN,...,0.848,1.129,3900.0,233.0,NaN,NaN,NaN,2981.0,NaN,46429.0
4,m24,3,23324.0,717.0,351.0,317.0,319.0,707.0,NaN,NaN,...,0.707,1.115,3754.0,203.0,NaN,NaN,NaN,3252.0,NaN,49402.0
1,sc,3,24577.0,790.0,375.0,341.0,307.0,735.0,NaN,NaN,...,0.836,1.163,3915.0,228.0,NaN,NaN,NaN,3155.0,NaN,45666.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11734,sc,7013,20375.4,1040.9,486.9,471.0,414.4,1070.5,341001.9532,374889.9263,...,NaN,NaN,3175.3,27.0,NaN,47964.0,7.925549e+05,1138.8,472929.9532,3639.0
11735,sc,7015,24160.0,938.4,416.7,445.7,495.2,1124.5,462206.2365,464788.2634,...,NaN,NaN,3838.3,16.7,NaN,59669.0,1.067078e+06,3390.1,642068.2365,17571.8
11736,sc,7020,24877.8,1002.2,514.6,529.2,659.2,1412.3,416042.0310,504676.4613,...,NaN,NaN,4387.4,34.5,NaN,56367.0,1.007722e+06,913.3,586964.0310,5012.8
11739,sc,7027,19930.3,900.6,387.3,345.1,425.6,955.7,394174.2868,368323.6123,...,NaN,NaN,3362.8,76.3,NaN,46926.0,8.541179e+05,1719.0,550395.2868,3392.8


In [122]:
unique_regions = regions_dict['Region'].unique().tolist()
mri = mri.loc[:, ~mri.columns.duplicated(keep='first')]
columns_to_keep = ['VISCODE2', 'RID'] + unique_regions
mri = mri[columns_to_keep]

In [123]:
viscode_counts = mri.groupby('RID')['VISCODE2'].count()

sorted_viscode_counts = viscode_counts.sort_values(ascending=False)

sorted_viscode_counts.head()

RID
1122    14
127     14
382     14
1300    13
1352    13
Name: VISCODE2, dtype: int64

In [124]:
rids_with_viscode_4_or_more = viscode_counts[viscode_counts >= 4].index

filtered_mri = mri[mri['RID'].isin(rids_with_viscode_4_or_more)]

filtered_mri.head()

,VISCODE2,RID,Brainstem,CorpusCallosumAnterior,CorpusCallosumCentral,CorpusCallosumMidAnterior,CorpusCallosumMidPosterior,CorpusCallosumPosterior,CorticalGM,CorticalWM,...,RightThalamus,RightUndetermined,RightVentralDC,RightVessel,RightWMHypoIntensities,SubcorticalGM,SupraTentorial,ThirdVentricle,TotalGM,WMHypoIntensities
1,sc,3,24577.0,790.0,375.0,341.0,307.0,735.0,NaN,NaN,...,7052.0,NaN,3915.0,228.0,NaN,NaN,NaN,3155.0,NaN,45666.0
2,m06,3,23784.0,774.0,378.0,358.0,314.0,701.0,NaN,NaN,...,6771.0,NaN,3951.0,220.0,NaN,NaN,NaN,3036.0,NaN,44708.0
3,m12,3,23759.0,763.0,369.0,323.0,307.0,721.0,NaN,NaN,...,6740.0,NaN,3900.0,233.0,NaN,NaN,NaN,2981.0,NaN,46429.0
4,m24,3,23324.0,717.0,351.0,317.0,319.0,707.0,NaN,NaN,...,6790.0,NaN,3754.0,203.0,NaN,NaN,NaN,3252.0,NaN,49402.0
5,sc,4,22145.0,1098.0,431.0,475.0,385.0,1047.0,NaN,NaN,...,6380.0,NaN,3589.0,42.0,NaN,NaN,NaN,1336.0,NaN,4959.0


In [125]:
filtered_mri.sort_values(by=['RID', 'VISCODE2'],inplace=True)

C:\Users\anisr\AppData\Local\Temp\ipykernel_8000\1522039963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_mri.sort_values(by=['RID', 'VISCODE2'],inplace=True)


In [126]:
desired_viscodes = ['sc', 'm06', 'm12', 'm24']

filtered_mri = filtered_mri[filtered_mri['VISCODE2'].isin(desired_viscodes)]

filtered_mri = filtered_mri.groupby('RID').filter(
    lambda x: set(x['VISCODE2']) == set(desired_viscodes))

filtered_mri.head()

,VISCODE2,RID,Brainstem,CorpusCallosumAnterior,CorpusCallosumCentral,CorpusCallosumMidAnterior,CorpusCallosumMidPosterior,CorpusCallosumPosterior,CorticalGM,CorticalWM,...,RightThalamus,RightUndetermined,RightVentralDC,RightVessel,RightWMHypoIntensities,SubcorticalGM,SupraTentorial,ThirdVentricle,TotalGM,WMHypoIntensities
2,m06,3,23784.0,774.0,378.0,358.0,314.0,701.0,NaN,NaN,...,6771.0,NaN,3951.0,220.0,NaN,NaN,NaN,3036.0,NaN,44708.0
3,m12,3,23759.0,763.0,369.0,323.0,307.0,721.0,NaN,NaN,...,6740.0,NaN,3900.0,233.0,NaN,NaN,NaN,2981.0,NaN,46429.0
4,m24,3,23324.0,717.0,351.0,317.0,319.0,707.0,NaN,NaN,...,6790.0,NaN,3754.0,203.0,NaN,NaN,NaN,3252.0,NaN,49402.0
1,sc,3,24577.0,790.0,375.0,341.0,307.0,735.0,NaN,NaN,...,7052.0,NaN,3915.0,228.0,NaN,NaN,NaN,3155.0,NaN,45666.0
11,m06,5,21374.0,992.0,446.0,463.0,383.0,1149.0,NaN,NaN,...,6395.0,NaN,3369.0,74.0,NaN,NaN,NaN,1493.0,NaN,5812.0


In [127]:
filtered_mri['VISCODE2'].value_counts()

m24    531
m12    524
sc     524
m06    522
Name: VISCODE2, dtype: int64

In [128]:
filtered_mri = filtered_mri.dropna(axis=1, how='all')
filtered_mri.head()

,VISCODE2,RID,Brainstem,CorpusCallosumAnterior,CorpusCallosumCentral,CorpusCallosumMidAnterior,CorpusCallosumMidPosterior,CorpusCallosumPosterior,Csf,RightParacentral,...,RightHippocampus,RightInferiorLateralVentricle,RightLateralVentricle,RightPallidum,RightPutamen,RightThalamus,RightVentralDC,RightVessel,ThirdVentricle,WMHypoIntensities
2,m06,3,23784.0,774.0,378.0,358.0,314.0,701.0,2022.0,3474.0,...,2471.0,6224.0,30036.0,1394.0,5550.0,6771.0,3951.0,220.0,3036.0,44708.0
3,m12,3,23759.0,763.0,369.0,323.0,307.0,721.0,2176.0,3234.0,...,2310.0,6501.0,30538.0,1521.0,5400.0,6740.0,3900.0,233.0,2981.0,46429.0
4,m24,3,23324.0,717.0,351.0,317.0,319.0,707.0,2023.0,3499.0,...,2269.0,7031.0,33017.0,1601.0,5688.0,6790.0,3754.0,203.0,3252.0,49402.0
1,sc,3,24577.0,790.0,375.0,341.0,307.0,735.0,1905.0,3539.0,...,2376.0,5921.0,28743.0,1692.0,5583.0,7052.0,3915.0,228.0,3155.0,45666.0
11,m06,5,21374.0,992.0,446.0,463.0,383.0,1149.0,1428.0,2710.0,...,3785.0,371.0,16661.0,1472.0,4667.0,6395.0,3369.0,74.0,1493.0,5812.0


In [129]:
print(filtered_mri.columns.tolist())

['VISCODE2', 'RID', 'Brainstem', 'CorpusCallosumAnterior', 'CorpusCallosumCentral', 'CorpusCallosumMidAnterior', 'CorpusCallosumMidPosterior', 'CorpusCallosumPosterior', 'Csf', 'RightParacentral', 'RightParahippocampal', 'RightParsOpercularis', 'RightParsOrbitalis', 'RightParsTriangularis', 'RightPericalcarine', 'RightPostcentral', 'RightPosteriorCingulate', 'RightPrecentral', 'RightPrecuneus', 'RightRostralAnteriorCingulate', 'RightRostralMiddleFrontal', 'RightSuperiorFrontal', 'RightSuperiorParietal', 'RightSuperiorTemporal', 'RightSupramarginal', 'RightTemporalPole', 'RightTransverseTemporal', 'RightUnknown', 'LeftInsula', 'RightInsula', 'LeftBankssts', 'LeftCaudalAnteriorCingulate', 'LeftCaudalMiddleFrontal', 'LeftCuneus', 'LeftEntorhinal', 'LeftFrontalPole', 'LeftFusiform', 'LeftInferiorParietal', 'LeftInferiorTemporal', 'LeftIsthmusCingulate', 'LeftLateralOccipital', 'LeftLateralOrbitofrontal', 'LeftLingual', 'LeftMedialOrbitofrontal', 'LeftMiddleTemporal', 'LeftParacentral', 'Le

In [130]:
filtered_mri['RID'].nunique()

522

In [131]:
null_columns = filtered_mri.columns[filtered_mri.isna().any()].tolist()
print(null_columns)

['FifthVentricle', 'RightVessel']


In [132]:
filtered_mri = filtered_mri.drop(columns=null_columns, axis=1)

In [133]:
# filtered_mri.columns.tolist()

In [134]:
filtered_mri.columns.tolist()

['VISCODE2',
 'RID',
 'Brainstem',
 'CorpusCallosumAnterior',
 'CorpusCallosumCentral',
 'CorpusCallosumMidAnterior',
 'CorpusCallosumMidPosterior',
 'CorpusCallosumPosterior',
 'Csf',
 'RightParacentral',
 'RightParahippocampal',
 'RightParsOpercularis',
 'RightParsOrbitalis',
 'RightParsTriangularis',
 'RightPericalcarine',
 'RightPostcentral',
 'RightPosteriorCingulate',
 'RightPrecentral',
 'RightPrecuneus',
 'RightRostralAnteriorCingulate',
 'RightRostralMiddleFrontal',
 'RightSuperiorFrontal',
 'RightSuperiorParietal',
 'RightSuperiorTemporal',
 'RightSupramarginal',
 'RightTemporalPole',
 'RightTransverseTemporal',
 'RightUnknown',
 'LeftInsula',
 'RightInsula',
 'LeftBankssts',
 'LeftCaudalAnteriorCingulate',
 'LeftCaudalMiddleFrontal',
 'LeftCuneus',
 'LeftEntorhinal',
 'LeftFrontalPole',
 'LeftFusiform',
 'LeftInferiorParietal',
 'LeftInferiorTemporal',
 'LeftIsthmusCingulate',
 'LeftLateralOccipital',
 'LeftLateralOrbitofrontal',
 'LeftLingual',
 'LeftMedialOrbitofrontal',
 

In [135]:
adnimerge = pd.read_csv("Imp_Files/ADNIMERGE_07Jan2024.csv")
adnimerge.head()

C:\Users\anisr\AppData\Local\Temp\ipykernel_8000\1902979561.py:1: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  adnimerge = pd.read_csv("Imp_Files/ADNIMERGE_07Jan2024.csv")


,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,...,NaN,1.33615,NaN,NaN,NaN,0.000000,0.00000,0,0,2023-07-07 04:59:40.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.000000,0.00000,0,0,2023-07-07 04:59:40.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.498289,5.96721,6,6,2023-07-07 04:59:40.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.999316,11.96720,12,12,2023-07-07 04:59:40.0
4,3,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,1.998630,23.93440,24,24,2023-07-07 04:59:40.0


In [136]:
adnimerge[adnimerge['RID'].isin(filtered_mri['RID'])]['DX_bl'].value_counts()

LMCI    2630
CN      2349
AD       554
Name: DX_bl, dtype: int64

In [137]:
adnimerge = adnimerge[['RID', 'VISCODE','DX_bl','AGE','PTGENDER','PTEDUCAT','PTETHCAT','PTRACCAT','PTMARRY','APOE4','DX','Month','Years_bl']]

In [138]:
adnimerge['Adjusted_Age'] = adnimerge['AGE'] + adnimerge['Years_bl']
adnimerge[adnimerge['RID'].isin(filtered_mri['RID'])]

,RID,VISCODE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,DX,Month,Years_bl,Adjusted_Age
1,3,bl,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,0,0.000000,81.300000
2,3,m06,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,6,0.498289,81.798289
3,3,m12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,12,0.999316,82.299316
4,3,m24,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,24,1.998630,83.298630
10,5,bl,CN,73.7,Male,16,Not Hisp/Latino,White,Married,0.0,CN,0,0.000000,73.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16323,127,m204,CN,70.6,Male,17,Not Hisp/Latino,White,Married,0.0,NaN,198,16.763900,87.363900
16330,734,m198,CN,72.6,Male,18,Not Hisp/Latino,White,Married,0.0,NaN,198,16.577700,89.177700
16331,1016,m198,CN,78.3,Female,13,Not Hisp/Latino,White,Widowed,0.0,NaN,198,16.375100,94.675100
16366,605,m204,CN,75.6,Female,16,Not Hisp/Latino,White,Married,1.0,Dementia,204,16.961000,92.561000


In [139]:
filtered_mri['VISCODE2'] = filtered_mri['VISCODE2'].replace('sc', 'bl')
filtered_mri.rename(columns={'VISCODE2':'VISCODE'}, inplace=True)
merged_file = pd.merge(filtered_mri, adnimerge, on=['RID', 'VISCODE'])

In [140]:
merged_file['RID'].nunique()

522

In [141]:
merged_file['VISCODE'].value_counts()

m24    531
m12    524
bl     524
m06    522
Name: VISCODE, dtype: int64

In [142]:
dx_changes = merged_file[['VISCODE', 'RID', 'DX_bl', 'DX']]

pivot_table = dx_changes.pivot_table(index=['RID', 'VISCODE'], values=['DX_bl', 'DX'], aggfunc='first')

pivot_table.reset_index()

,RID,VISCODE,DX,DX_bl
0,3,bl,Dementia,AD
1,3,m06,Dementia,AD
2,3,m12,Dementia,AD
3,3,m24,Dementia,AD
4,5,bl,CN,CN
...,...,...,...,...
2083,1425,m24,MCI,LMCI
2084,1427,bl,MCI,LMCI
2085,1427,m06,MCI,LMCI
2086,1427,m12,Dementia,LMCI


In [143]:
pivot_table['DX'] = pivot_table['DX'].replace('Dementia', 'AD')

changes = pivot_table[pivot_table['DX'] != pivot_table['DX_bl']]

changes.reset_index()

,RID,VISCODE,DX,DX_bl
0,22,m24,MCI,CN
1,30,bl,MCI,LMCI
2,30,m06,AD,LMCI
3,30,m12,AD,LMCI
4,30,m24,AD,LMCI
...,...,...,...,...
967,1425,m24,MCI,LMCI
968,1427,bl,MCI,LMCI
969,1427,m06,MCI,LMCI
970,1427,m12,AD,LMCI


In [144]:
changes = changes.reset_index()

In [145]:
merged_file

,VISCODE,RID,Brainstem,CorpusCallosumAnterior,CorpusCallosumCentral,CorpusCallosumMidAnterior,CorpusCallosumMidPosterior,CorpusCallosumPosterior,Csf,RightParacentral,...,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,DX,Month,Years_bl,Adjusted_Age
0,m06,3,23784.0,774.0,378.0,358.0,314.0,701.0,2022.0,3474.0,...,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,6,0.498289,81.798289
1,m12,3,23759.0,763.0,369.0,323.0,307.0,721.0,2176.0,3234.0,...,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,12,0.999316,82.299316
2,m24,3,23324.0,717.0,351.0,317.0,319.0,707.0,2023.0,3499.0,...,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,24,1.998630,83.298630
3,bl,3,24577.0,790.0,375.0,341.0,307.0,735.0,1905.0,3539.0,...,Male,18,Not Hisp/Latino,White,Married,1.0,Dementia,0,0.000000,81.300000
4,m06,5,21374.0,992.0,446.0,463.0,383.0,1149.0,1428.0,2710.0,...,Male,16,Not Hisp/Latino,White,Married,0.0,CN,6,0.501027,74.201027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,bl,1425,18744.0,502.0,274.0,292.0,296.0,738.0,1081.0,2759.0,...,Female,16,Not Hisp/Latino,White,Divorced,0.0,MCI,0,0.000000,75.600000
2097,m06,1427,18996.0,1001.0,392.0,438.0,348.0,902.0,1446.0,3287.0,...,Female,12,Not Hisp/Latino,White,Widowed,0.0,MCI,6,0.479124,70.079124
2098,m12,1427,18506.0,917.0,363.0,388.0,391.0,937.0,1470.0,3352.0,...,Female,12,Not Hisp/Latino,White,Widowed,0.0,Dementia,12,1.004790,70.604790
2099,m24,1427,17978.0,924.0,383.0,415.0,356.0,939.0,1381.0,3343.0,...,Female,12,Not Hisp/Latino,White,Widowed,0.0,Dementia,24,1.995890,71.595890


In [146]:
medhist = pd.read_csv("Imp_Files/RECMHIST_25Aug2023.csv")
medhist.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,RECNO,EXAMDATE,MHNUM,MHDESC,MHDTONSET,MHCUR,MHSTAB,update_stamp
0,ADNI1,12,2,107,sc,sc,17-08-2005,NaN,1.0,17-08-2005,9.0,--/--/1984 Pancreatitis,NaN,0.0,-4.0,00:00.0
1,ADNI1,14,2,107,sc,sc,17-08-2005,NaN,2.0,17-08-2005,9.0,"--/--/2001 Non Insulin dependent diabetes, die...",NaN,1.0,1.0,00:00.0
2,ADNI1,16,2,107,sc,sc,17-08-2005,NaN,3.0,17-08-2005,3.0,--/--/1993 cataracts removed from both eyes,NaN,0.0,-4.0,00:00.0
3,ADNI1,18,1,10,f,f,18-08-2005,NaN,1.0,18-08-2005,3.0,"astigmatism OU since 1950, corrective lenses w...",NaN,1.0,1.0,00:00.0
4,ADNI1,20,1,10,f,f,18-08-2005,NaN,2.0,18-08-2005,6.0,Hepatitis A 1990,NaN,0.0,-4.0,00:00.0


In [147]:
medhist = medhist[['RID', 'VISCODE2', 'MHDESC']]
medhist = medhist.dropna(subset=['MHDESC'])

In [148]:
# medhist = medhist[medhist['RID'].isin(filtered_mri['RID'])]

In [149]:
import re

sleep_keywords = [
    r'\bsleep\b', r'\binsomnia\b', r'\bapnea\b', r'\bsleep apnea\b',
    r'\brestless\b', r'\bnarcolepsy\b', r'\bdisturbance\b',
    r'\bsleep disorder\b', r'\bhypersomnia\b', r'\bdyssomnia\b',
    r'\bnocturnal\b', r'\bsomnambulism\b', r'\bparasomnia\b'
]

sleep_pattern = re.compile('|'.join(sleep_keywords), re.IGNORECASE)

In [150]:
medhist_sleep = medhist[medhist['MHDESC'].str.contains(
    sleep_pattern, na=False)]

In [151]:
medhist_sleep_ = medhist_sleep[medhist_sleep['RID'].isin(filtered_mri['RID'])]
medhist_sleep_['RID'].nunique()

78

In [152]:
merged_file[merged_file['RID'].isin(medhist_sleep_['RID'])].groupby('DX_bl')['RID'].nunique()


DX_bl
AD       6
CN      26
LMCI    46
Name: RID, dtype: int64

In [153]:
merged_file[(merged_file['RID'].isin(medhist_sleep_['RID'])) & (merged_file['VISCODE'] == 'm24')].groupby("DX")['RID'].nunique()

DX
CN          27
Dementia    16
MCI         35
Name: RID, dtype: int64

In [154]:
merged_file[(merged_file['RID'].isin(medhist_sleep_['RID'])) & (
    merged_file['VISCODE'] == 'm12')].groupby("DX")['RID'].nunique()

DX
CN          26
Dementia    11
MCI         41
Name: RID, dtype: int64

In [155]:
merged_file[(merged_file['RID'].isin(medhist_sleep_['RID'])) & (
    merged_file['VISCODE'] == 'm06')].groupby("DX")['RID'].nunique()

DX
CN          26
Dementia     6
MCI         46
Name: RID, dtype: int64

In [156]:
medical_his = pd.read_csv("Files/MEDHIST_19Feb2025.csv")
medical_his = medical_his[['RID','VISCODE2','MHPSYCH','MH2NEURL','MH4CARD','MH5RESP','MH19OTHR']]

In [157]:
medi_his_sc = medical_his[(medical_his['RID'].isin(filtered_mri['RID'])) & (medical_his['VISCODE2']=='sc')]

In [158]:
merged_file = pd.merge(merged_file, medi_his_sc.drop(['VISCODE2'],axis=1), on=['RID'])

In [159]:
medhist_sleep_

,RID,VISCODE2,MHDESC
415,72,sc,probable idiopathic restless leg syndrome - pr...
682,127,sc,"vertigo 09/unk/2000, obstructive sleep apnea 0..."
701,128,sc,Obstructive Sleep Apnea 2004
927,158,sc,"TAKES TRANXENE x 10 YEARS FOR ""NERVES"", SLEEP."
966,210,sc,restless leg syndrome 10/unk/2004
...,...,...,...
27902,1045,m72,sleep apnea
27903,1045,m72,restless leg syndrome
28236,919,m72,Possible history of sleep apnea- not using CPAP
28730,120,m84,Insomnia


In [160]:
merged_file

,VISCODE,RID,Brainstem,CorpusCallosumAnterior,CorpusCallosumCentral,CorpusCallosumMidAnterior,CorpusCallosumMidPosterior,CorpusCallosumPosterior,Csf,RightParacentral,...,APOE4,DX,Month,Years_bl,Adjusted_Age,MHPSYCH,MH2NEURL,MH4CARD,MH5RESP,MH19OTHR
0,m06,3,23784.0,774.0,378.0,358.0,314.0,701.0,2022.0,3474.0,...,1.0,Dementia,6,0.498289,81.798289,0,0,1,0,0
1,m12,3,23759.0,763.0,369.0,323.0,307.0,721.0,2176.0,3234.0,...,1.0,Dementia,12,0.999316,82.299316,0,0,1,0,0
2,m24,3,23324.0,717.0,351.0,317.0,319.0,707.0,2023.0,3499.0,...,1.0,Dementia,24,1.998630,83.298630,0,0,1,0,0
3,bl,3,24577.0,790.0,375.0,341.0,307.0,735.0,1905.0,3539.0,...,1.0,Dementia,0,0.000000,81.300000,0,0,1,0,0
4,m06,5,21374.0,992.0,446.0,463.0,383.0,1149.0,1428.0,2710.0,...,0.0,CN,6,0.501027,74.201027,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,bl,1425,18744.0,502.0,274.0,292.0,296.0,738.0,1081.0,2759.0,...,0.0,MCI,0,0.000000,75.600000,1,0,1,0,0
2097,m06,1427,18996.0,1001.0,392.0,438.0,348.0,902.0,1446.0,3287.0,...,0.0,MCI,6,0.479124,70.079124,0,0,1,1,0
2098,m12,1427,18506.0,917.0,363.0,388.0,391.0,937.0,1470.0,3352.0,...,0.0,Dementia,12,1.004790,70.604790,0,0,1,1,0
2099,m24,1427,17978.0,924.0,383.0,415.0,356.0,939.0,1381.0,3343.0,...,0.0,Dementia,24,1.995890,71.595890,0,0,1,1,0


In [161]:
medhist_participants = medhist[(medhist['RID'].isin(merged_file['RID'])) & (medhist['VISCODE2']=='sc')]

In [162]:
merged_file.groupby('DX')['RID'].nunique()

DX
CN          189
Dementia    201
MCI         248
Name: RID, dtype: int64

In [163]:
region_list = [
    "RightEntorhinal",
    "RightTemporalPole",
    "RightParahippocampal",
    "RightInferiorTemporal",
    "RightMiddleTemporal",
    "RightFusiform",
    "RightInferiorParietal",
    "RightIsthmusCingulate",
    "RightBankssts",
    "RightPrecuneus",
    "RightHippocampus",
    "RightAmygdala",
    "RightAccumbensArea",
    "RightMedialOrbitofrontal",
    "RightPallidum",
    "RightCaudalMiddleFrontal",
    "RightPutamen",
    "RightRostralAnteriorCingulate",
    "RightParacentral",
    "RightPrecentral",
    "RightLingual",
    "LeftEntorhinal",
    "LeftHippocampus",
    "LeftFusiform",
    "LeftMiddleTemporal",
    "LeftParacentral",
    "RightInferiorLateralVentricle",
    "LeftInferiorLateralVentricle",
    "RightLateralVentricle",
    "LeftLateralVentricle",
    "Icv",
]
others = ['RID', 'VISCODE', 'DX_bl', 'DX', 'Adjusted_Age', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4', 'MHPSYCH', 'MH2NEURL', 'MH4CARD', 'MH5RESP', 'MH19OTHR']

In [165]:
merged_updated = merged_file[region_list + others]

In [102]:
merged_file.to_csv("Volumes_Merge.csv", index=False)

In [166]:
merged_file.groupby('MHPSYCH')['RID'].nunique()

MHPSYCH
0    364
1    158
Name: RID, dtype: int64

In [167]:
merged_file['VISCODE'].value_counts()

m24    531
m12    524
bl     524
m06    522
Name: VISCODE, dtype: int64

In [168]:
psychom = pd.read_csv("Files/UWNPSYCHSUM_13Feb2025.csv")
psychom = psychom[['RID','VISCODE2','ADNI_MEM','ADNI_EF','ADNI_VS','ADNI_LAN','ADNI_EF2']]
psychom = psychom.rename(columns={'VISCODE2':'VISCODE'})

In [169]:
psychom[psychom['RID'].isin(merged_file['RID'])]['RID'].nunique()

522

In [170]:
merged_file = pd.merge(merged_updated, psychom, on=['RID','VISCODE'])

In [171]:
merged_file.to_csv("Volumes_Merge_new_regions.csv", index=False)

In [68]:
merged_file.columns.tolist()

['VISCODE',
 'RID',
 'Brainstem',
 'CorpusCallosumAnterior',
 'CorpusCallosumCentral',
 'CorpusCallosumMidAnterior',
 'CorpusCallosumMidPosterior',
 'CorpusCallosumPosterior',
 'Csf',
 'RightParacentral',
 'RightParahippocampal',
 'RightParsOpercularis',
 'RightParsOrbitalis',
 'RightParsTriangularis',
 'RightPericalcarine',
 'RightPostcentral',
 'RightPosteriorCingulate',
 'RightPrecentral',
 'RightPrecuneus',
 'RightRostralAnteriorCingulate',
 'RightRostralMiddleFrontal',
 'RightSuperiorFrontal',
 'RightSuperiorParietal',
 'RightSuperiorTemporal',
 'RightSupramarginal',
 'RightTemporalPole',
 'RightTransverseTemporal',
 'RightUnknown',
 'LeftInsula',
 'RightInsula',
 'LeftBankssts',
 'LeftCaudalAnteriorCingulate',
 'LeftCaudalMiddleFrontal',
 'LeftCuneus',
 'LeftEntorhinal',
 'LeftFrontalPole',
 'LeftFusiform',
 'LeftInferiorParietal',
 'LeftInferiorTemporal',
 'LeftIsthmusCingulate',
 'LeftLateralOccipital',
 'LeftLateralOrbitofrontal',
 'LeftLingual',
 'LeftMedialOrbitofrontal',
 '

In [71]:
sleep_sev = pd.read_csv("Files/All_Subjects_NPIQ_07Feb2025.csv")[['RID','VISCODE2','NPIK','NPIKSEV']]

In [72]:
sleep_sev_part = sleep_sev[(sleep_sev['RID'].isin(merged_file['RID'])) & (sleep_sev['VISCODE2'].isin(['bl','m06','m12','m24']))]

In [73]:
min_count = 522
merged_file_filt = merged_file.groupby('VISCODE').apply(lambda x: x.sample(n=min_count, random_state=1)).reset_index(drop=True).sort_values(by=['RID','VISCODE'])

In [74]:
combined_sleep_df = pd.merge(
    medhist_participants,
    sleep_sev_part[['RID', 'NPIK', 'NPIKSEV']],
    on='RID',
    how='left'
)

In [75]:
combined_sleep_df

,RID,VISCODE2,MHDESC,NPIK,NPIKSEV
0,3,sc,1995-hypertension; 1995-hyperlipidemia,0.0,-4.0
1,3,sc,1995-hypertension; 1995-hyperlipidemia,0.0,-4.0
2,3,sc,1995-hypertension; 1995-hyperlipidemia,0.0,-4.0
3,3,sc,1995-hypertension; 1995-hyperlipidemia,0.0,-4.0
4,3,sc,2000-osteoarthritis,0.0,-4.0
...,...,...,...,...,...
18443,467,sc,diarrhea onset --/--/1991,0.0,-4.0
18444,824,sc,mild depression,1.0,1.0
18445,824,sc,mild depression,1.0,1.0
18446,824,sc,mild depression,1.0,1.0


In [76]:
sleep_sev_part

,RID,VISCODE2,NPIK,NPIKSEV
1,3,bl,0.0,-4.0
2,5,bl,0.0,-4.0
5,16,bl,0.0,-4.0
7,30,bl,0.0,-4.0
8,22,bl,0.0,-4.0
...,...,...,...,...
3521,941,m24,1.0,2.0
3571,1205,m24,-1.0,-4.0
3583,1408,m24,0.0,-4.0
3816,1352,m24,0.0,-4.0


In [77]:
sleep_patterns = {
    'Sleep_Apnea': [
        r'\bsleep apnea\b', r'\bOSA\b', r'\bCPAP\b',
        r'\bobstructive sleep apnea\b', r'\bobstructive\b'
    ],
    'Restless_Legs': [
        r'\bRLS\b', r'\brestless legs\b', r'\brestless leg syndrome\b',
        r'\bleg movements\b', r'\bperiodic limb\b'
    ],
    'Insomnia': [
        r'\binsomnia\b', r'\bsleep difficulty\b', r'\bsleep onset\b',
        r'\bsleep maintenance\b', r'\bpoor sleep\b', r'\btrouble sleeping\b'
    ],
    'Sleep_Disturbance_Other': [
        r'\bsleep disturbance\b', r'\bhypersomnia\b', r'\bsomnambulism\b',
        r'\bparasomnia\b', r'\bnocturnal\b', r'\bdyssomnia\b', r'\bsleep disorder\b'
    ]
}


def classify_sleep_conditions(desc, sleep_patterns):
    sleep_status = {condition: 0 for condition in sleep_patterns.keys()}

    for condition, patterns in sleep_patterns.items():
        for pattern in patterns:
            if re.search(pattern, str(desc), re.IGNORECASE):
                sleep_status[condition] = 1
                break  

    return pd.Series(sleep_status)


sleep_df = combined_sleep_df.apply(
    lambda row: classify_sleep_conditions(row['MHDESC'], sleep_patterns), axis=1)

combined_sleep_df = pd.concat([combined_sleep_df, sleep_df], axis=1)

In [78]:
sleep_unique_df = combined_sleep_df.groupby('RID').agg({
    'VISCODE2': 'first',  
    'MHDESC': lambda x: '; '.join(x.dropna().unique()),
    'NPIK': 'max',  
    'NPIKSEV': 'max',  
    'Sleep_Apnea': 'max',
    'Restless_Legs': 'max',
    'Insomnia': 'max',
    'Sleep_Disturbance_Other': 'max'
}).reset_index()

In [79]:
def classify_sleep_quality(row):
    sleep_conditions = row['Sleep_Apnea'] + row['Restless_Legs'] + \
        row['Insomnia'] + row['Sleep_Disturbance_Other']

    if sleep_conditions > 0 or (row['NPIK'] == 1.0 and row['NPIKSEV'] >= 2.0):
        return 'High Risk of Sleep Disturbance'
    elif row['NPIK'] == 1.0 and row['NPIKSEV'] == 1.0:
        return 'Mild/Moderate Sleep Disturbance'
    else:
        return 'No Significant Sleep Disturbance'

In [80]:
sleep_unique_df['Sleep_Quality'] = sleep_unique_df.apply(
    classify_sleep_quality, axis=1)
sleep_unique_df

,RID,VISCODE2,MHDESC,NPIK,NPIKSEV,Sleep_Apnea,Restless_Legs,Insomnia,Sleep_Disturbance_Other,Sleep_Quality
0,3,sc,1995-hypertension; 1995-hyperlipidemia; 2000-o...,0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance
1,5,sc,2000 glaucoma; 1941-Rosacea; 1995-ulcer; smoke...,0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance
2,10,sc,2004- depressive symptoms; 2000-alzheimer dise...,0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance
3,14,sc,Depression in 1988. Treated with antidepressan...,0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance
4,16,sc,"2003 glaucoma; 1994 asthma, 1998 collapsed lun...",0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance
...,...,...,...,...,...,...,...,...,...,...
517,1414,sc,Chronic hearing loss (has VA benefits); s/p bi...,0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance
518,1418,sc,Hypertension since 08/--/2002; Carpal Tunnel s...,0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance
519,1419,sc,5 basal cell lesions removed between 1997 and ...,1.0,1.0,0,0,0,0,Mild/Moderate Sleep Disturbance
520,1425,sc,"Mild depressiona and anxiety, started around 1...",0.0,-4.0,0,0,0,0,No Significant Sleep Disturbance


In [81]:
sleep_unique_df.groupby('Sleep_Quality')['RID'].nunique()

Sleep_Quality
High Risk of Sleep Disturbance      119
Mild/Moderate Sleep Disturbance      96
No Significant Sleep Disturbance    307
Name: RID, dtype: int64

In [82]:
sleep_conditions_cols = [
    'Sleep_Apnea', 'Restless_Legs', 'Insomnia', 'Sleep_Disturbance_Other'
]

baseline_df_med = sleep_unique_df[['RID', 'MHDESC'] + sleep_conditions_cols]

In [83]:
longitudinal_df_sleep = pd.merge(sleep_sev_part, baseline_df_med, on='RID', how='left').sort_values(by=['RID','VISCODE2'])

In [84]:
def update_sleep_quality(row):
    sleep_conditions = row['Sleep_Apnea'] + row['Restless_Legs'] + \
        row['Insomnia'] + row['Sleep_Disturbance_Other']

    if sleep_conditions > 0 or (row['NPIK'] == 1.0 and row['NPIKSEV'] >= 2.0):
        return 'High Risk of Sleep Disturbance'
    elif row['NPIK'] == 1.0 and row['NPIKSEV'] == 1.0:
        return 'Mild/Moderate Sleep Disturbance'
    else:
        return 'No Significant Sleep Disturbance'


longitudinal_df_sleep['Sleep_Quality'] = longitudinal_df_sleep.apply(
    update_sleep_quality, axis=1)

In [124]:
longitudinal_df_sleep.to_csv("Longitudinal sleep.csv", index=False)

In [125]:
merged_file_filt.to_csv("Volumes_Merge.csv", index=False)

In [85]:
len(set(sleep_unique_df['RID']).intersection(set(longitudinal_df_sleep['RID'])))

522

In [87]:
vitals = pd.read_csv("Files/All_Subjects_VITALS_19Feb2025.csv")

In [88]:
unique_rids = vitals['RID'].unique()

In [89]:
filtered_data = vitals[(vitals['VSWEIGHT'] > 0) & (vitals['VSHEIGHT'] > 0)]

viscode2_per_rid = filtered_data.groupby(
    'RID')['VISCODE2'].first().reset_index()

In [90]:
def calculate_bmi(weight, weight_unit, height, height_unit):
    if weight_unit == 2:
        weight = weight * 0.453592
    if height_unit == 1:
        height = height * 0.0254
    if height > 0: 
        return weight / (height ** 2)
    return None

valid_data = vitals[(vitals['VSWEIGHT'] > 0) & (vitals['VSHEIGHT'] > 0)]

valid_data['BMI'] = valid_data.apply(lambda row: calculate_bmi(
    row['VSWEIGHT'], row['VSWTUNIT'], row['VSHEIGHT'], row['VSHTUNIT']), axis=1)

bmi_per_rid = valid_data.groupby('RID').apply(
    lambda group: group.loc[group.index[0], ['RID', 'BMI']]).reset_index(drop=True)

C:\Users\anisr\AppData\Local\Temp\ipykernel_8000\2765008809.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['BMI'] = valid_data.apply(lambda row: calculate_bmi(


In [91]:
first_valid_rows = valid_data.sort_values(
    'VISCODE2').groupby('RID').first().reset_index()
bmi_per_rid = first_valid_rows[['RID', 'BMI']]

In [92]:
bmi_per_rid.describe()

,RID,BMI
count,3954.000000,3954.000000
mean,4421.770106,39.195366
std,3067.203784,29.624162
min,2.000000,0.000751
25%,1190.250000,0.005518
50%,4632.500000,51.714461
75%,6492.750000,61.183349
max,10498.000000,118.993687


In [96]:
others

['RID',
 'VISCODE',
 'DX_bl',
 'DX',
 'Adjusted_Age',
 'PTGENDER',
 'PTEDUCAT',
 'PTETHCAT',
 'PTRACCAT',
 'PTMARRY',
 'APOE4',
 'MHPSYCH',
 'MH2NEURL',
 'MH4CARD',
 'MH5RESP',
 'MH19OTHR']

In [100]:
merged_table = merged_file_filt[region_list + others + ['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN', 'ADNI_EF2']]

In [99]:
bmi_per_rid 

,RID,BMI
0,2,61.322478
1,3,10.927769
2,4,59.903501
3,5,12.273373
4,6,53.626655
...,...,...
3949,10485,48.179619
3950,10490,9.803460
3951,10494,75.638118
3952,10497,0.008033


In [102]:
table_all = merged_table.merge(bmi_per_rid, on='RID') 

In [14]:
bmi_per_rid.to_csv("Updated_BMI.csv", index=False)

In [103]:
table_all.to_csv("Table_All.csv", index=False)

In [110]:
table_all 

,RightEntorhinal,RightTemporalPole,RightParahippocampal,RightInferiorTemporal,RightMiddleTemporal,RightFusiform,RightInferiorParietal,RightIsthmusCingulate,RightBankssts,RightPrecuneus,...,MH2NEURL,MH4CARD,MH5RESP,MH19OTHR,ADNI_MEM,ADNI_EF,ADNI_VS,ADNI_LAN,ADNI_EF2,BMI
0,985.0,1341.0,1881.0,8355.0,10387.0,8005.0,11917.0,2204.0,2887.0,8112.0,...,0,1,0,0,-1.021,-1.144,-0.880,-1.368,-1.062,10.927769
1,1183.0,1159.0,1994.0,8612.0,9311.0,7010.0,11950.0,2168.0,2893.0,8089.0,...,0,1,0,0,-1.187,-0.674,-1.221,-1.638,-0.670,10.927769
2,834.0,1350.0,1776.0,7542.0,9736.0,7624.0,11785.0,2002.0,2750.0,7580.0,...,0,1,0,0,-0.914,-0.969,-0.548,-1.837,-1.034,10.927769
3,545.0,1279.0,1701.0,7534.0,9316.0,7264.0,11152.0,2129.0,2770.0,7697.0,...,0,1,0,0,-1.369,-1.511,0.739,-1.700,-1.605,10.927769
4,2525.0,1999.0,2305.0,9870.0,11579.0,13203.0,14826.0,2096.0,2497.0,9306.0,...,0,0,0,0,0.743,0.165,0.739,0.259,-0.152,12.273373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,1215.0,1979.0,1336.0,8891.0,10183.0,5573.0,12743.0,1993.0,1656.0,7274.0,...,0,1,0,0,-1.661,-1.078,0.739,-2.517,-0.826,0.004549
2084,1834.0,2097.0,2006.0,9236.0,12498.0,8556.0,13878.0,2043.0,2166.0,7649.0,...,0,1,1,0,-0.366,-1.746,-1.530,-0.492,-2.042,72.926002
2085,1574.0,2117.0,2007.0,9070.0,12815.0,8297.0,13912.0,2144.0,2229.0,7920.0,...,0,1,1,0,-1.445,-1.642,-1.813,-1.640,-1.788,72.926002
2086,1739.0,2208.0,2007.0,8904.0,12154.0,8450.0,13366.0,2116.0,2148.0,8087.0,...,0,1,1,0,-1.764,-1.625,-0.656,-0.946,-2.047,72.926002


In [ ]:
baseline_df = table_all[table_all['VISCODE'] == 'bl']

exclude_cols = ['RID', 'VISCODE', 'DX_bl', 'DX', 'PTGENDER', 'PTETHCAT', 'PTRACCAT', 'PTMARRY']
numeric_cols = [col for col in baseline_df.columns if baseline_df[col].dtype in [float, int] and col not in exclude_cols]

summary = {}
for dx in baseline_df['DX_bl'].unique():
    group = baseline_df[baseline_df['DX_bl'] == dx]
    summary[dx] = [
        f"{group[col].mean():.2f} ± {group[col].std():.2f}" if group[col].notnull().any() else "NA"
        for col in numeric_cols
    ]

summary_table = pd.DataFrame(summary, index=numeric_cols)
summary_table

,AD,CN,LMCI
RightEntorhinal,1381.24 ± 437.27,1928.01 ± 384.51,1679.18 ± 438.44
RightTemporalPole,1726.17 ± 386.29,1860.72 ± 334.98,1862.04 ± 374.66
RightParahippocampal,1646.16 ± 323.07,1953.77 ± 311.31,1820.19 ± 307.80
RightInferiorTemporal,8130.09 ± 1877.34,9507.91 ± 1608.28,9050.09 ± 1774.08
RightMiddleTemporal,9143.99 ± 1767.78,10407.56 ± 1472.43,9973.31 ± 1730.92
RightFusiform,7317.55 ± 1360.75,8450.97 ± 1237.75,8056.42 ± 1293.04
RightInferiorParietal,11028.38 ± 1903.51,12577.96 ± 1767.45,12170.17 ± 2142.35
RightIsthmusCingulate,1870.58 ± 392.47,1979.86 ± 298.00,1949.43 ± 355.95
RightBankssts,1947.17 ± 510.99,2144.13 ± 388.23,2011.35 ± 420.45
RightPrecuneus,7246.06 ± 1203.22,7880.24 ± 1016.16,7767.19 ± 1112.01
